In [0]:
!apt-get install nvidia-cuda-toolkit
!pip3 install numba


In [0]:
import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/lib/nvidia-cuda-toolkit/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/lib/x86_64-linux-gnu/libnvvm.so"

In [0]:
import matplotlib.pyplot as plt
import matplotlib.patches as ptc
from random import uniform
import numba

from numba import cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
import numpy as np

In [0]:
from numba import cuda
print(cuda.gpus)

<Managed Device 0>


In [0]:
@cuda.jit()
def mcCount(rng_states, iterations, out):
    
    thread_id = cuda.grid(1)
    
    inside = 0
    for i in range(iterations):
        x = xoroshiro128p_uniform_float32(rng_states, thread_id)
        y = xoroshiro128p_uniform_float32(rng_states, thread_id)
        if x**2 + y**2 <= 1.0:
            inside += 1

    out[thread_id] = 4.0 * inside / iterations

In [0]:
import time
threads_per_block = 64
blocks = 100
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=time.time())
out = np.zeros(threads_per_block * blocks, dtype=np.float32)
points = np

mcCount[blocks, threads_per_block](rng_states, 1000000, out)
print('pi:', out.mean())

pi: 3.141597
